https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

Research Paper https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf

Sample code Encoder code - https://www.geeksforgeeks.org/word-embedding-using-universal-sentence-encoder-in-python/

In [3]:
pip install "tensorflow>=2.0.0"

In [5]:
pip install tensorflow-hub

In [2]:
import tensorflow_hub as hub

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [20]:
import numpy as np

In [4]:
# Load pre-trained universal sentence encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [5]:
Sentences = [
    "How old are you",
    "What is your age",
    "I love to watch Television",
    "I am wearing a wrist watch"
]
embeddings = embed(Sentences)

In [7]:
emb_1 = embed([Sentences[0]])

In [9]:
embeddings

<tf.Tensor: shape=(4, 512), dtype=float32, numpy=
array([[-0.06045125, -0.00204541,  0.02656925, ...,  0.00764413,
        -0.02669661,  0.05110302],
       [-0.08415681, -0.08687922,  0.03446117, ..., -0.01439389,
        -0.04546221,  0.03639964],
       [ 0.0816019 , -0.01570276, -0.05659244, ..., -0.07133698,
         0.11040761, -0.0071095 ],
       [-0.00369539,  0.03064634, -0.05556112, ...,  0.01751423,
         0.0316496 , -0.05139377]], dtype=float32)>

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
train_data = pd.read_csv('gdrive/My Drive/train_2024.csv', quoting=3)
eval_data = pd.read_csv('gdrive/My Drive/test_2024.csv', quoting=3)
dev_data = pd.read_csv('gdrive/My Drive/dev_2024.csv', quoting=3)

In [13]:
#Data Split - Random Seed = 42 is important.
#USE embeddings are saved according to the data generated by the seed.

train_data = pd.read_csv('gdrive/My Drive/train_2024.csv', quoting=3)
eval_data = pd.read_csv('gdrive/My Drive/test_2024.csv', quoting=3)
dev_data = pd.read_csv('gdrive/My Drive/dev_2024.csv', quoting=3)


train_texts = train_data['text'].tolist()
dev_texts = dev_data['text'].tolist()
eval_texts = eval_data['text'].tolist()

train_labels = train_data['label'].tolist()
dev_labels = dev_data['label'].tolist()
eval_labels = eval_data['label'].tolist()

train_texts.extend(dev_texts)
train_labels.extend(dev_labels)

X = train_texts
y = train_labels

train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.1, random_state=42)
len(train_texts), len(train_labels), len(eval_texts), len(eval_labels)

In [14]:
len(X), len(y), len(eval_texts), len(eval_labels)

(110000, 110000, 12001, 12001)

In [15]:
train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.1, random_state=42)
len(train_texts), len(train_labels), len(eval_texts), len(eval_labels)

(99000, 99000, 12001, 12001)

In [16]:
type(train_texts)

list

In [17]:
train_texts_embedded = []
for txt in train_texts:
  train_texts_embedded.append(embed([txt]))

In [44]:
val_texts_embedded = []
for txt in val_texts:
  val_texts_embedded.append(embed([txt])[0])


eval_texts_embedded = []
for txt in eval_texts:
  eval_texts_embedded.append(embed([txt])[0])

In [45]:
val = np.array(val_texts_embedded)

In [46]:
val.shape

(11000, 512)

In [47]:
eval = np.array(eval_texts_embedded)
eval.shape

(12001, 512)

In [48]:
np.savetxt("gdrive/My Drive/USE_eval.csv", eval, delimiter=",")

In [50]:
np.savetxt("gdrive/My Drive/USE_val.csv", val, delimiter=",")

In [21]:
t = np.array(train_texts_embedded)

In [22]:
t.shape

(99000, 1, 512)

In [38]:
t_2 = []
for x in train_texts_embedded:
  t_2.append(x[0])

In [39]:
t = np.array(t_2)
t.shape

(99000, 512)

In [43]:
np.savetxt("gdrive/My Drive/USE_train.csv", t, delimiter=",")